# Einführung & Grundlagen Machine Learning (ML)  

    18. & 19. März 2020, München    
    Autor: Andreas Barth, barth@strategiepilot
***

Willkommen in **Ihrem persönlichen Jupyter-Notebook**.
  
Sie können in diesem Notebook alle Beispiele live nachvollziehen, aber auch eigene Varianten ausprobieren.  
In der Menüleiste finden sich die wichtigsten Funktionen für "Maus"-User.  
Hier noch einige sehr hilfreiche Tastatur-Kürzel für effizientes Arbeiten mit der Tastatur:

* **Ausführen/Run** einer Zelle mit ... [SHIFT+ENTER]
* Eine neue leere Zelle **über** einer Zelle einfügen mit ... [a] 
* Eine neue leere Zelle **unter** einer Zelle einfügen mit ... [b]
* Eine Zelle **löschen/entfernen** !!VORSICHT!! mit ... [dd]
* Eine Zelle in **Markdown-Format** umwandeln mit ... [m]
* Eine Zelle in **Coding-Format** umwandeln mit ... [y]

In [ ]:
a = 3
b = 8
c = a+b
d = a*b
e = d/c

In [ ]:
txt = "Guten Morgen"
txt

Diesen Code müssen wir am Anfang IMMER ausführen:

In [ ]:
%matplotlib inline

# Grundausstattung an Bibliotheken, die wir immer laden
import numpy as np                  # Numerische Operationen, Lineare Algebra
from scipy.stats import *           # Funktionsbibliothek mit statistischen Funktionen
import matplotlib.pyplot as plt     # Funktionsbilio<thek zur Visualisierung von Daten/Ergebnissen
import pandas as pd                 # Bearbeitung von tabellarischen Daten (sog. Data Frames)
import seaborn as sns               # Erweiterte Visualisierung von Daten/Ergebnissen etc.
import warnings                     # Ermöglicht die Deaktivierung von best. Warnmeldungen
import random                       # Damit kann man Zufallszahlen generieren
import os                           # Ermöglicht Zugriff auf das Dateiablagesystem 
import datetime as dt               # Funktionsbiliothek zum Arbeiten mit Zeitreihen Daten
import pickle                       # Ermöglicht das Abspeichern von Objekten (z.B. trainierten Modellen)

# Ein paar Einstellungen, die einem das Leben einfacher machen
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = [8, 4]
from IPython.core.pylabtools import figsize
plt.style.use('seaborn-white')
# sns.set_style('white')
# sns.set_context('talk')


In [ ]:
# os.chdir("F://Data/Data Science Uni/40200/BMW Bank Seminar/Data Sets")

***
#  Inhalt

2. Daten & Preprocessing
   - Daten laden u. explorieren
   - Einfaches Preprocessing


3. Unsupervised Learning
   - Clustering mit Kmeans


4. Supervised Learning: Lineare Modelle
   - Datenset: BMW Pricing Challenge
   - Lineare Regression


5. Supervised Learning: Classification
   - Datenset: "Give me some Credit"
   - Ein erstes Modell: Decision Tree
   - Modellauswahl & -beurteilung verschiedener Modelle 


6. Exkurs: NLP Natural Language Processing & Text Mining

***
## 2. Daten & Preprocessing

### 2.1 Daten laden & explorieren
Zunächst schauen wir uns an, wie man Daten in Python gut einlesen, explorieren und für ML vorbereiten kann.  
Wenn man mit tabellarischen Daten arbeiten möchte bietet sich insbesondere die Funktionsbibliothek PANDAS an.  
Da wir sie bereits standardmässig (s.o. bei imports) aufgerufen haben, steht sie uns sofort zur Verfügung.

Wir laden ein Datenset mit 261 PKW Modellen, die mit jeweils 8 Merkmalen beschrieben werden.  
Im Urzustand sind die Daten so noch nicht in dem Format, dass wir für ML brauchen.  
Darum kümmern wir uns jetzt ...

In [ ]:
cars = pd.read_csv("cars.csv", sep=",", decimal=".")     # Einlesen der.csv Datei vom Verzeichnis und in den Dataframe "cars" schreiben
cars.head()                                              # .head()  zeigt die ersten 5 Datensätze/Zeilen des Dataframes an 

Umfang unseres df bestimmen:  Anzahl Datensätze (Zeilen), Anzahl Features (Spalten)

In [ ]:
cars.shape

Anzeigen der Mermale / Features / Spalten, ihrer Datentypen und Anzahl von fehlenden Werten

In [ ]:
cars.info()

Mittelwerte aller Merkmale ermitteln mit .mean()  
Das funktioniert aber auch mit ...    
.median()  
.std()  
.var()  
.min()  
.max()  

In [ ]:
cars.mean()

Oder wenn man nur die Werte eines bestimmten Merkmals ermitteln möchte:

In [ ]:
cars.cubicinches.mean()

Noch einfacher ... eine komplette Beschreibung der Verteilungsparameter aller numerischen Merkmale unseres Datensets

In [ ]:
cars.describe().T

Visuelle Exploration der Daten geht auch:

(a) Histogramm

In [ ]:
figsize(20,10)             # stellt die Größe der Abbildung ein (Horizontale, Vertikale)
_= cars.hist(bins=30,)     # erzeugt ein Histogramm mit 30er Intervallschritten, einstellbar über bins=xx

(b) Countplot

In [ ]:
# Visualidierung Anzahl Modelle nach "Country"
figsize(5,2)  
_= sns.countplot(x=cars.country, data=cars, )

(c) Boxplot

In [ ]:
# Visualidierung Boxplot: Verteilung der Kubikinches nach "Country"
figsize(5,4)  
_= sns.boxplot(x=cars.country, y=cars.cubicinches, data=cars)

(d) Violinplot

In [ ]:
# Visualidierung Violinplot: Verteilung der Kubikinches nach "Country"
figsize(5,4)  
_= sns.violinplot(x=cars.country, y=cars.cubicinches, data=cars)

(e) Etwas advancend: Paarweise Verteilung ausgewählter Feature

In [ ]:
figsize(20,20)
_= sns.pairplot(data=cars, vars=["mpg","cylinders","hp","time-to-60"], size=3)  # hue=cars.country

### 2.2 Wichtige Preprocessing Schritte
Leider sind Rohdaten in der Realität selten (oder nie) in einem für ML Algorithmen geigneten Zustand,  
so dass ein PreProcessing und Vorbereiten der Daten erforderlich ist.
Die gängigsten Arbeitsschritte sind ...

+ Fehlende Werte ersetzen oder bereinigen
+ Kategorielle Daten encoden (umwandeln)
+ Numerische Merkmale standardisieren / skalieren


#### *Fehlende Werte*
... heißen in Python meistens "NA" (oder nan). Viele ML Algorithmen funktioneren nicht mit NA Werten im Datenset.  
Welche Strategien kann man anwenden?  
+ Löschen von einzelnen Datensätzen mit NA Werten
+ Löschen von einzelnen Merkmalen (Feature) mit NA Werten
+ NA Werte durch Schätzwerte ersetzen => Mittelwert, Median, Modus, Max-Wert, Min-Wert, individueller Wert, Regressionsmodell lernen

Wie sieht es in unserem Datenset aus?  
Welche Merkmale haben NA und wieviele davon?

In [ ]:
cars.isna().sum()
# oder prozentual:  
# cars.isna().mean()

Da uns prozentual nur wenige Werte fehlen, können wir sie bedenkenlos mit dem jeweiligen Mittelwert oder Median des Merkmals ersetzen

In [ ]:
cars = cars.fillna(cars.mean())   # alternativ mit .median()

Sind jetzt alle fehlenden Werte ersetzt worden?

In [ ]:
# OK, dann kümmern wir uns noch um das nicht-numerische Merkaml "country"
cars.country = cars.country.fillna("MISSING")

Verteilung des Merkmals "country"

In [ ]:
cars.country.value_counts()

#### *Kategorielle alphanumerische Daten umwandeln*
Sehr viele ML Algorithmen (fast alle in der Bibliothek Scikit-Learn) können nur numerische Daten verarbeiten.  
In der Praxis sind kategorielle Merkmale aber häufig alphanumerisch: Farbe, Geschlecht, Hersteller, Modell, Land ...  
Wenn man diese Merkmale als Feature nutzen möchte, muss man sie in eine numerische Form encoden:
  
Zwei gängige Methoden dafür sind "Label Encoding" und "One Hot Encoding".  

OH Encoding hat ggü. Label Encoding einen entscheidenden Vorteil:  
Label Encoding stellt eine (häufig nicht real existierende) Logik bzw. Rangfolge zwischen den Merkmalen her:  
Label Encoding unseres Merkmals Country führt zu: (0, US), (1, Europe), (2, Japan). Ist Japan > Europe > US ??    
Beim OH Encoding hingegen werden die Merkmale transformiert, ohne dass eine ungewollte Rangfolge der Ausprägungen ensteht.

Wir transformieren also unser Feature "colour" mit dem OH-Encoder:

In [ ]:
cars_target = cars.country.copy()          # Brauchen wir später noch ...

# Jetzt transformieren wir cars mit "One Hot Encoding"
cars = pd.get_dummies(cars, )
cars.sample(4)

#### *Daten normalisieren/standardisieren*
Sehr viele Algorithmen nutzen mathematische Distanzmaße wie z.B. den Abstand eines Datenpunktes vom Mittelwert.  
Wenn die einzelnen Feature in ihren Ausprägungen unterschiedlich stark skalieren (z.B. Anzahl Zylinder und PS)  
dann "verzerren" diese unterschiedlichen Skalen die Ergebnisse des Algorithmus.

Lösungsstrategie: Einheitliche Skalierung der Daten, d.h. man standardisiert sie.  
Schauen wir uns die statistischen Eckwerte (Lageparameter) unserer numerischen Feature an:

In [ ]:
feat_num = ['mpg', 'cylinders', 'cubicinches', 'hp', 'weightlbs', 'time-to-60']   # Liste feat_num := Vereinfacht die Adressierung
cars.loc[:, feat_num].describe()[1:3]

Wir standardisieren unsere Daten mit der sog. Z-Score Methode (Normalisierung)  
Die Funktionsbibliothek Scikit-Learn (ML Methoden) bietet dafür eine geeignete Methode an.

In [ ]:
from sklearn.preprocessing import StandardScaler    # importieren des Tools aus scikit-learn

X = cars.copy()                        # jetzt wandeln wir unseren Dataframe in eine Datenmatrix X um
X = X[feat_num]                        # wir skalieren nur die ersten 6 Feature (nicht das Jahr und die Länder)

scaler = StandardScaler().fit(X)       # Trainiert den Scaler auf die Datenmatrix
X = scaler.transform(X)                # Transformiert Datenmatrix X

print(cars.loc[:0,feat_num])           # Ausgabe der ersten Zeile des Cars Datensets
print(X[:1])                           # Ausgabe der ersten Zeile der transformierten Matrix X
print(80*"-")
for i in X[:5]: print("\n",i)          # Pretty Printing der ersten 5 transformierten Datensätze


***
Nach der Exploration und Vorbereitung unserer Daten wenden wir uns jetzt dem ML zu:

## 3. Unsupervised Learning: Clustering mit k-Means
Wir arbeiten mit unseren cars Daten weiter. Beim **"unsupervised" Learning"** wird ein Modell **ohne ein vorhandenes Label (Lernsignal)** trainiert.  
D.h. in unserem Beispiel, dass wir simulieren die Informtion der Herkunft "country" nicht zu besitzen.  
Dafür erstellen wir eine Datenmatrix X des cars-Datenset OHNE das Feature "country".  
Wir versuchen das Herkunftsland (Region) über k-Means zu bestimmen:

In [ ]:
X = cars[feat_num].copy()  # Datenmatrix X mit unseren Features
X.sample(3)

Jetzt wenden wir den K-Means Algorithmus an, um die Daten zu clustern.  
Bei K-Means muss man die Anzahl der "vermuteten" Cluster dem Algorithmus vorgeben:

In [ ]:
X = cars[feat_num].copy()                  # Datenmatrix X mit unseren Features
from sklearn.cluster import KMeans         # Import des Algorithmus
# X = StandardScaler().fit_transform(X)    # Standardisiert die Datenmatrix - lassen wir erstmal weg
km = KMeans(n_clusters=3).fit(X)           # Wendet k-Means auf X an, mit Vorgabe 3 Cluster

Mal sehen wie gut k-Means auf unserem Datenset funktioniert.  
In realita würden wir natürlich die "richtige" Verteilung nicht kennen ...

In [ ]:
print("Gruppierung durch k-Means Algo:\n", pd.Series(km.labels_).value_counts())
print()
print("Reale Verteilung im Datenset\n", cars_target.value_counts())
# km.labels_  

Wir visualisieren die k-Means Ergebnisse zum besseren Verständnis.  
Diese Merkmale stehen uns zur Verfügung:

In [ ]:
list(enumerate(cars[feat_num].columns))

In [ ]:
# Visualisierung der Zuordnungen
figsize(10,7)

# Hier können wir die Merkmale für das Plotting auswählen
x,y = 0,2         # x = Merkmal X-Achse, y = Merkmal Y-Achse 

plt.scatter(X.iloc[:,x], X.iloc[:,y], c=km.labels_,  cmap="viridis")
plt.scatter(km.cluster_centers_[:,x], km.cluster_centers_[:,y], c='tomato', marker='*', s=300, ) # Scatterplot mit Centroids
plt.title(f"k-Means Clustering mit Feature {cars.columns[x]} & {cars.columns[y]}", fontsize=15)
plt.xlabel(cars.columns[x]); plt.ylabel(cars.columns[y]); plt.show()

In [ ]:
cars[cars.mpg>35].sort_values("mpg", ascending=False)[:10]

Wenn wir mit der Qualität unseres Modells zufrieden wären (ohne Kenntnis der Echtdaten schwierig!).  
Könnten wir es nun verwenden, um weitere NEUE Datensätze zu beurteilen:

In [ ]:
pkw_new = [
    [31.4,4,85,65,2500,19,],
    [16,8,304,150,4200,12,],
    [24,4,113,95,2000,16,],
    [24,4,107,90,2750,15,],
    [37.2,4,86,65,2019,16,],
    [21.5,4,121,110,2600,13,]] 

# Wenn wir auf normalisierten Daten trainiert haben, müssen wir die Daten jetzt auch normalisieren:
# pkw_new = scaler.transform(pkw_new)

km.predict(pkw_new)

***
## 4. Supervised Learning: Regression
Reminder: Supervised Learning, d.h. Modelle werden **immer anhand der vorhandenen Lerninformation (Target Variable)** trainiert.

### 4.1 Datenset: BMW-PRICING CHALLENGE

Dafür bearbeiten wir jetzt ein praxisnäheres Beispiel: Das BMW-Pricing Challenge Datenset auf der Plattform KAGGLE  

https://www.kaggle.com/danielkyrka/bmw-pricing-challenge 

Die Autoren dieses Datensets schreiben dazu:

* With this challenge we hope to [...] gain some insight in what the main factors are that drive the value of a used car.  
* The data provided consists of almost 5000 real BMW cars that were sold via a b2b auction in 2018.
* The price shown in the table is the highest bid that was reached during the auction.
* We have also extracted 8 criteria based on the equipment of car that we think might have a good impact on the value of a used car.
* These criteria have been labeled feature1 to feature 8 and are shown in the data below.

In [ ]:
# Zunächst laden wir die Rohdaten wieder aus unserem Verzeichnis
bmw = pd.read_csv("bmw_pricing_challenge.csv")
bmw.sample(3)

In [ ]:
bmw.info()

*Summary:*
* Keine NA Werte
* 5 Kategorielle Merkmale (Datentyp: "Object")
* 3 Numerische Merkmale (Ganzzahlig: Datentyp "Integer")
* 2 Merkmale mit Datumsinformationen (im "falschen" Datenformat "Object")
* 8 "anonyme" Merkmale mit Datentyp Bool ("True" vs. "False")

Zunächst bearbeiten wir die Datums-Informationen und "bauen" daraus weitere Feature:

In [ ]:
# Die beiden Datums-Merkmale 'sold_at' und 'registration_date' sollten wir besser in ein Datetime-Format konvertieren
bmw.registration_date = pd.to_datetime(bmw.registration_date)
bmw.sold_at = pd.to_datetime(bmw.sold_at)

Jetzt können wir das "Alter" der Fahrzeuge i.S. der Differenz als zusätzliches Feature einbauen.  
Da alle Auktionen aus dem Jahr 2018 sind, spielt das Verkaufsjahr keine Rolle, aber vielleicht der Monat der Auktion?

In [ ]:
# Neue Datums-Features ableiten
bmw["period"] = bmw.sold_at - bmw.registration_date   # erstellt Spalte mit Differenz in Tagen
bmw["period"] = bmw.period.dt.days                    # normiert die Differenz in Tageseinheiten
bmw["Sell_Month"] = bmw.sold_at.dt.month              # Der Monat, in dem die Auktion stattfand


In [ ]:
bmw.head()

In [ ]:
# Noch ein kurzer Blick auf die Verteilung der numerischen Variablen ...
bmw.describe().T

Schauen wir uns noch kurz die Verteilung des Fahrzeugalters an:

In [ ]:
# Verteilung des Fahrzeugalters (in Jahren) im Datenset:
_= (bmw.period/365).hist(bins=70, figsize=(8,5))

Betrachten wir die kategoriellen Features noch etwas genauer: 

In [ ]:
for i in ["model_key", "fuel", "paint_color", "car_type"]:
    print()
    print(f"Merkmal {i}, Anzahl der Ausprägungen {bmw[i].nunique()}:\n")
    print(bmw[i].value_counts())
    print("-"*80)

Insbesondere bei den Modellvarianten (Feature 'model_key') gibt es sehr viele Ausprägungen.   
Lässt sich das "vereinfachen"?

In [ ]:
figsize(8,6)
titel = f"Anteil der 20 häufigst vertretenen Modelle: {bmw.model_key.value_counts(normalize=True)[:20].sum().round(2)}"
_= bmw.model_key.value_counts(normalize=True)[:20].apply(lambda x: x*100).round(2).sort_values(ascending=True).plot(kind="barh", fontsize=12, title=titel)
plt.xlabel("%"); plt.show()

Wir beschränken unser Modell auf diese 20 am häufigsten vorkommenden Modellreihen.  
Dafür erstellen wir ein zweites Datenset "bmwSmall" in dem nur noch diese Fahrzeugreihen enthalten sind.

In [ ]:
t20_models = bmw.model_key.value_counts()[:20].index.to_list()  # Auslesen der T20 Modellbezeichnungen
bmwSmall = bmw.loc[bmw.model_key.isin(t20_models),:].copy()     # Neuer DataFrame bmwSmall mit Filterung auf die T20 Modelle
print(bmwSmall.shape)                                           # Umfang des neuen DataFrame
bmwSmall.model_key.value_counts()                               # In bmwSmall sind nur noch die T20 Modelle

Und schauen uns die Verteilung der erzielten Auktionspreise je nach Modell an:

In [ ]:
bmwSmall.columns

In [ ]:
figsize(15,10)
x,y = bmwSmall.model_key, bmwSmall.price
_= sns.boxplot(x, y, data=bmwSmall, color="tomato") # violinplot
plt.title("Verteilung der Fahrzeugpreise nach Modellreihen")
plt.xticks(fontsize=14, rotation=80); plt.xlabel("Modellreihe"), plt.ylabel("Preis"); plt.ylim(0,70_000); plt.show()

In [ ]:
bmwByModel = bmwSmall.groupby("model_key")
bmwByModel.price.describe()

In [ ]:
# SavePickle("bmwSmall", bmwSmall)

In [ ]:
# Gepickelten DF einlesen
# FILE = "bmwSmall"
# open_df = open(FILE+'.pickle','rb')
# data = pickle.load(open_df)
# open_df.close()

# bmwSmall = data.copy()
# bmwSmall.shape

### 4.2 Lineare Regression

Jetzt bauen wir unsere Datenmatrix auf, auf der wir dann das Regressionsmodell trainieren wollen.  
Die Arbeitspakete:

    (1) Datenmatrix und Targetvektor aufbauen: Auswahl der Feature, die wir mit ins Modell nehmen möchten  
    (2) NA Werte bereinigen => Es gibt keine in diesem Datenset ... entfällt  
    (3) OH-Encoding für die kategoriellen Daten  
    (4) Standardisieren der numerischen Daten  
  
    (5) Trainingsset und Testset trennen  
    (6) Lineares Regressionsmodell traineren  
    (7) Regressionsmodell visualisieren  

##### (1) Datenmatrix & Targetvektor
Wir wählen wir aus, welche Features wir in das Modell "mitnehmen" möchten: 

In [ ]:
features = ['model_key', 'mileage', 'engine_power','fuel', 'paint_color', 'car_type',
            'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8',
            'period', 'Sell_Month', ]

Mit dieser Auswahl erstellen wir eine Feature-Matrix X und einen Targetvektor y

In [ ]:
bmwSmall.reset_index(inplace=True)   # Numerischen Index neu aufbauen (Lücken aus dem Filterprozess schließen!)
X = bmwSmall[features].copy()
y = bmwSmall.price.copy()
print(X.shape)
print(y.shape)

##### (3) & (4) OH-Encoding und Standardisieren

In [ ]:
# Wir importieren die Preprocessing Tools aus Scikit-Learn
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler   # Unsere Werkzeuge

# Wir legen ein paar Listen an, um das PreProcessing zu erleichtern
feat_cat = ["model_key", "fuel", "paint_color", "car_type", ] 
feat_num = ['mileage', 'engine_power', 'period',]
feat_bool = ['feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_6','feature_7', 'feature_8']
feat_other = ['Sell_Month']

# Jetzt vereinzeln wir die Matrix X in vier Teil-Matrizen 
Xcat = X[feat_cat] 
Xnum = X[feat_num]
Xbool = X[feat_bool]
Xother = X[feat_other]

# OH-Encoding auf der Matrix mit den kategoriellen Daten
oh = OneHotEncoder(sparse=False)
Xcat = oh.fit_transform(Xcat)
Xcat_cols = oh.get_feature_names(feat_cat)
Xcat = pd.DataFrame(data=Xcat, columns=Xcat_cols)

# # Alternativ: Label-Encoding auf der Matrix mit den kategoriellen Daten
# le = LabelEncoder()
# Xcat = Xcat.apply(le.fit_transform)
# Xcat = pd.DataFrame(data=Xcat, columns=feat_cat)

# Standardisieren auf der Matrix mit den numerischen Daten
# scaler = StandardScaler()
# Xnum = scaler.fit_transform(Xnum)
# Xnum = pd.DataFrame(Xnum, columns=feat_num)

# Zusammenführen der vier Teilmatrizen zu einer Datenmatrix X
X = pd.concat([Xcat, Xnum, Xbool, Xother], axis=1,  )

print(f"Featurematrix X mit {X.shape[0]} Datensätzen und {X.shape[1]} Feature/Variablen")
print(f"Targetvektor y mit {y.shape[0]} Datensätzen")

In [ ]:
X.head(3)

##### (5) Trainings- & Testset splitten
Wir splitten in ein Trainingsset mit 70% fürs Training und 30% fürs Testen 

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=42)

##### (6) Lineares Regressionsmodell trainieren 

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression().fit(X_train, y_train)  # Model instanziieren und auf die Trainingsdaten trainieren

scoreTrain = lr.score(X_train, y_train)        # Ermittelt R² Score für Trainingsdaten
scoreTest = lr.score(X_test, y_test)           # Ermittelt den R² für die Testdaten

print("-"*65)
print(f"Anteil der erklärbaren Varianz, R² auf dem Trainingsset = {scoreTrain:.2f}")
print(f"Anteil der erklärbaren Varianz, R² auf den TESTDATEN (!) = {scoreTest:.2f}")
print("-"*65)

In [ ]:
# Ausgabe der einzelnen Faktoren mit ihren Gewichten in der Regression:
weights = pd.Series(lr.coef_, index=X.columns.to_list(),)
weights.sort_values(ascending=False)

Mit unserem Modell können wir jetzt den Preis für "neue" ungesehene Daten schätzen:  
Zur Vereinfachung ziehen wir uns aus unseren "unberührten" Testdaten ein Sample und lassen es durch unser Modell schätzen:

In [ ]:
Size = 5
Sample = X_test.sample(Size, random_state=815)
yreal = pd.Series(y_test[Sample.index])
ypred = pd.Series(lr.predict(Sample), index=Sample.index, name="price_pred").astype("int")
result = pd.concat([ypred,yreal,Sample], axis=1)
result.T

## 5 Supervised Learning: Classification

### 5.1 KAGGLE Competition - "Give Me Some Credit"
https://www.kaggle.com/c/GiveMeSomeCredit/data

Das schreiben die Autoren auf KAGGLE:

*Credit scoring algorithms, which make a guess at the probability of default, are the method banks use to determine whether or not a loan should be granted.  
This competition requires participants to improve on the state of the art in credit scoring, by predicting the probability that somebody will experience financial distress in the next two years.*

*The goal of this competition is to build a model that borrowers can use to help make the best financial decisions.*

Hier eine kurze Beschreibung der einzelnen Variablen:

In [ ]:
pd.set_option('display.max_colwidth', -1)
cs_info = pd.read_excel("cs-Data Dictionary.xls", header=1); cs_info

In [ ]:
# Datenset laden
cs = pd.read_csv("cs-training-small.csv")
cs = cs.iloc[:,1:]
cs.info()

Jetzt werfen für mal einen Blick auf die Verteilung der Werte der einzelnen Variablen ...

In [ ]:
cs.describe(percentiles=[.05,.25,.5,.75,.95], ).T

Wir sehen an dieser kurzen Statistik bereits, dass in diesem Datenset furchbar große Ausreißer enthalten sind (siehe z.B. RUUL und DebtRation).  
Wir wissen nicht, ob diese "Ausreißer" wichtig sind für unser Modell ... oder ob es z.B. vernachlässigbare Eingabe-/Übertragungsfehler sind?  
Betrachten wir, wieviel Anteil diese speziellen Datensätze an unserer durch das Modell zu prognostizierenden Variable "SeriousDlqin2yrs" haben:

In [ ]:
csRUUL_o1 = cs.loc[(cs.RevolvingUtilizationOfUnsecuredLines > 1.0),:]  # Auslesen der RUUL auffälligen Datensätze
csDebtRatio_o1 =  cs.loc[(cs.DebtRatio > 1.0),:]                       # Auslesen der DebtRatio auffälligen Datensätze
RUUL_Defaults = csRUUL_o1.SeriousDlqin2yrs.sum()                       # Anzahl der Defaults in den RUUL auffälligen Datensätzen
DebtR_Defaults = csDebtRatio_o1.SeriousDlqin2yrs.sum()                 # Anzahl der Defaults in den DebtRatio auffälligen Datensätzen

# Ausgabe der Berechnungen
print(f"Anzahl der 'Defaults' im gesamten Datenset {cs.SeriousDlqin2yrs.sum()}, entspricht {cs.SeriousDlqin2yrs.mean()}")
print("-"*100)
print(f"Anzahl der 'Auffälligen' RUULs Datensätze: {csRUUL_o1.shape[0]}")
print(f"Anzahl der 'Defaults' in den 'Auffälligen' RUULs Datensätzen {RUUL_Defaults} entspricht {RUUL_Defaults/csRUUL_o1.shape[0]}")
print("-"*100)
print(f"Anzahl der 'Auffälligen' DebtRatio Datensätze: {csDebtRatio_o1.shape[0]}")
print(f"Anzahl der 'Defaults' in den 'Auffälligen' DebtRatio Datensätzen {DebtR_Defaults} entspricht {DebtR_Defaults/csDebtRatio_o1.shape[0]}")

Zumindest die RUULs liefern einen überdurchschnittlichen Erklärungsbeitrag für unser Modell.  
Wir nehmen die auffälligen Merkmale mit in unsere weiteren Überlegungen.  
Jetzt bauen wir unsere Datenmatrix X und unseren Targetvektor y.

In [ ]:
X = cs.iloc[:,:-1].copy()
y = cs.iloc[:,-1]
print(X.shape,y.shape)
print(f"Anteil Defaults im gesamten Datenset {y.mean():.3f}")

Wir splitten in ein Trainingsset (2/3) und ein Testset (1/3):

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33333, shuffle=True, stratify=y, random_state=123)
print(f' Trainingsset: {X_train.shape, y_train.shape} / Test Set: {X_test.shape, y_test.shape}')

### 5.2 Classification mit Decision Tree Model

In [ ]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve

model = 'Decision Tree'
t_names = ['Kein Default', 'Default']

estimator = DecisionTreeClassifier(class_weight="balanced", ) # max_depth=5
estimator.fit(X_train, y_train)

ytrue = y_test
ypred = estimator.predict(X_test)

accuracy = accuracy_score(ytrue, ypred)
roc_auc = roc_auc_score(ytrue, ypred)
print(f"Dummy-Baseline Accuracy: {1-y_test.mean()}")
print(f'Accuracy Score: {accuracy:.4f}, AUC: {roc_auc:.4f}')
print("\n",classification_report(ytrue, ypred, target_names=t_names))

# Feature Importance aus Model in Dataframe FI schreiben
fi_data = {'Feature': list(X_train.columns), 'F_Importance': estimator.feature_importances_}
FI = pd.DataFrame(data=fi_data)
FI = FI.sort_values('F_Importance', ascending=False); FI

# Confusion Matrix erstellen
mat = confusion_matrix(ytrue, ypred,)
print("Confusion Matrix:\n",mat)

Mit ein paar Optimierungen können wir bereits moderate/gute Ergebnisse erzielen.
Nach diesen ersten "Gehversuchen" schicken wir ein paar weitere Modelle ins Rennen:

### 5.3 Classification mit verschiedenen Modellen

Beim Decision Tree Classifier ist es nicht notwendig die Daten zu standardisieren.  
Bei den Modellen, die wir jetzt zusätzlich ins Spiel bringen, könnte es sehr hilfreich sein.  
Wir behalten uns diesen Preprocessing-Schritt noch vor und probieren es zunächst ohne Standardisierung.

In [ ]:
# # Standardisieren auf der Matrix mit den numerischen Daten
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.fit_transform(X_test)

Wir bauen uns ein Pipeline aus verschiedenen Classifiern, die wir in einem "Durchgang" auf unsere Trainings- und Testdaten anwenden werden.  
Die einzelnen Schritte:

+ Importieren der notwendigen Classifier Alogrithmen u. verschd. Werkzeuge.
+ Instanziierung der einzelnen Algorithmen (so wird ein konkretes Learner-Objekt daraus).
+ Erstellen einer Pipeline (Festlegen, welche Modelle tatsächlich angewendet werden sollen).
+ Anlegen eines Dataframe, um die Ergebnisse der einzelnen Modelle abzuspeichern.
+ Pipeline-Logik: Ruft die vorab defierten Classifier auf u. wendet sie auf X_train u. X_test an.
+ Ausgeben der Ergebnisse aus unserem Dataframe

In [ ]:
# Importieren der Classifier Algorithmen, die wir als Kandidaten verwenden möchten:
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Importieren von Metriken und Zeitfunktionen
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score, classification_report
import time

# Hier sind unsere Classifier Kandidaten Modelle
clf1 = GaussianNB()
clf2 = SVC(class_weight="balanced",)
clf3 = LogisticRegression(class_weight="balanced")
clf4 = KNeighborsClassifier()
# Dem Random Forest spendieren wir 3 Varianten ...
clf5 = RandomForestClassifier(class_weight="balanced", n_jobs=-1)
clf6 = RandomForestClassifier(n_estimators = 300, class_weight="balanced", max_depth=3,  bootstrap=True, n_jobs=-1)
clf7 = RandomForestClassifier(n_estimators = 500, class_weight="balanced", max_depth=5,  bootstrap=False, n_jobs=-1)

# Das ist unsere Pipeline die wir durchlaufen
pipeline = [(1, "NB",clf1),
           (2, "SVC", clf2),
           (3, "LogReg", clf3),
           (4, "Knn5", clf4),
           (5, "RF", clf5),
           (6, "RF opt1", clf6),
           (7, "RF opt2", clf7),
          ]  
# Wir speichern die "Rundenergebnisse" der einzelnen Classifier in einem Dataframe
results = pd.DataFrame( {"Estimator":[], "Accuracy":[], "Precision":[], "Recall":[], "f1":[], "AUC":[], "Duration":[]} )
models_fitted = []  # Ablegen der gefitteten Modelle (Objekte) in einer Liste

# Durchlauf mehrerer Modelle und Wegschreiben des Ergebnisses
for i, name, estimator in pipeline:
    
    # Model fitten u. in Liste ablegen
    start = time.time()                     # Stoppuhr: Zwischenzeit nehmen
    est = estimator.fit(X_train, y_train)   # model aus Listing nehmen und fitten
    models_fitted.append(est)

    # Scorings erstellen
    ytrue = y_test                          # ...
    ypred = est.predict(X_test)             # model auf Testdaten anwenden (predict)
    
    acc = accuracy_score(ytrue, ypred )     # Accuracy 
    prec = precision_score(ytrue, ypred )   # Precision 
    rec = recall_score(ytrue, ypred,  )     # Recall
    f1 = f1_score(ytrue, ypred, )           # f1-Score
    auc = roc_auc_score(ytrue, ypred, )     # AUC
    end = time.time()                       # Stoppuhr: Zwischenzeit nehmen
    duration = end - start                  # Walltime in Variable abspeichern
    
    results.loc[i,:] = [name, acc, prec, rec, f1, auc, duration]
    
print(f"Dummy-Baseline Accuracy: {1-y_test.mean()}")
results.round(3)

In [ ]:
results_not_normalized.round(3)

In [ ]:
results_not_normalized = results.copy()

## Exkurs: Textmining

# Parking / Backstage

In [ ]:
def SavePickle(name, object):
    import pickle
    save_df = open(str(name)+'.pickle','wb')
    pickle.dump(object,save_df)
    save_df.close()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test_total, y_train, y_test_total = train_test_split(X, y, test_size=0.4, shuffle=True, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test_total, y_test_total, test_size=0.5, shuffle=True, stratify=y_test_total, random_state=42)
print(f' Trainingsset: {X_train.shape, y_train.shape} / Validation Set: {X_val.shape, y_val.shape}  / Test Set: {X_test.shape, y_test.shape}')

In [ ]:
# Confusion Matix plotten
def Plot_confMatrix(y_real, y_pred, title='Titel'):
    '''
    Erstellen einer Confusion Matrix Grafik
    im Abgleich von Label y_real und Prognose y_pred
    '''
    from sklearn.metrics import confusion_matrix
    mat = confusion_matrix(y_real, y_pred)
    # sns.set(font_scale=1.4)
    sns.heatmap(mat, square=True, annot=True,  cmap='Blues', cbar=False, ) #, fmt='d'
    plt.xlabel('Prediction')
    plt.ylabel('True value')
    plt.title(title);
    return plt.show()   

In [ ]:
## Schrittweise Selektion von Features

results = pd.DataFrame({"AnzFeature":[], "Feature":[], "Score":[]})

for i in range(4,len(X_train.columns)):
    columns=X_train.columns.to_list()
    cs = []
    for _ in range(i): 
        c = max([(lr.fit(X_train[cs+[c]],y_train).score(X_test[cs+[c]],y_test),c) for c in columns])[1]
        columns.remove(c)
        cs.append(c)
    score = lr.score(X_test[cs],y_test).round(5)
    results.loc[i,:] = [int(i), cs, score]
        
#     print(cs)
#     score = lr.score(X_test[cs],y_test); score.round(5)

results = results.sort_values(by="Score", ascending=False)
results[:1]

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
category_names = ["Kein Default", "Default"]
sns.heatmap(mat, annot=True, fmt="d", cmap="Blues", cbar=False,
            xticklabels=category_names, yticklabels=category_names)
plt.ylabel("Actual")
plt.xlabel("Predicted"); plt.show()